In [6]:

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

#warnings.filterwarnings(action='once')
import pandas as pd
from pandas import MultiIndex, Int16Dtype

import numpy as np
import math

from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, f1_score
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV, PredefinedSplit
# models
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.svm import LinearSVR
import xgboost as xgb

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

from sklearn import ensemble
from sklearn import datasets
from sklearn import model_selection

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

In [7]:
#!pip3 install xgboost --upgrade

In [8]:
df_train = pd.read_csv('../data/train_encoded_1.csv')
df_test = pd.read_csv('../data/test_encoded_1.csv')
df_test = df_test.reindex(columns=df_train.columns.intersection(df_test.columns))

In [45]:
from sklearn.model_selection import train_test_split
train, valid = train_test_split(df_train, test_size=0.2)

In [46]:
X_final = df_train.drop(['price', 'price_per_sqft'], axis=1)
y_final = train['price_per_sqft']
X_train = train.drop(['price', 'price_per_sqft'], axis=1)
y_train = train['price_per_sqft']

X_valid = valid.drop(['price', 'price_per_sqft'], axis=1)
y_valid = valid['price_per_sqft']

df_test_size_sqft = df_test['size_sqft'].copy()

np.random.seed(0)
idxs = np.arange(X_train.shape[0])
np.random.shuffle(idxs)
df_test.columns.difference(X_train.columns)


Index([], dtype='object')

In [13]:
xgb_model = xgb.XGBRegressor(eval_metric=["error"], verbose_eval=50, seed =42)
xgb_model.fit(X_train, y_train)

yt_pred = xgb_model.predict(X_train)
yv_pred = xgb_model.predict(X_valid)

print('Training RMSE: {:.3f}'.format(mean_squared_error(yt_pred, y_train, squared=False)))
print('Test RMSE: {:.3f}'.format(mean_squared_error(yv_pred, y_valid, squared=False)))



[19:09:36] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:573: 
Parameters: { "verbose_eval" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Training RMSE: 142.696
Test RMSE: 228.356


In [ ]:
yTest = xgb_model.predict(df_test)

yTest = yTest * df_test_size_sqft
yTest = pd.DataFrame(yTest.values, columns=['Predicted'], index=df_test.index)

yTest.to_csv('../predictions/xgboost_grid_search_plain.csv')

In [17]:
warnings.filterwarnings('ignore')

xgb_model = xgb.XGBRegressor(verbose_eval=50, seed =42)

parameters = {'objective':['reg:squarederror'],
              'eta': [0.03], #so called `eta` value
              'gamma':[0],
              'max_depth': [6],
              'min_child_weight':[1],
              'max_delta_step':[0],
              'subsample': [1],
              'sampling_method':['uniform'],
              'colsample_by*':[1],
              'n_estimators': [2500, 3000],
              'reg_alpha':[0],
              'reg_lambda': [1],
              'tree_method':['auto'],
              'max_leaves':[0],
              'num_parallel_tree':[1],
              'booster': ['gbtree'], 
              'eval_metric': ['rmse'], 
              'verbose_eval': [50],
                'seed' :[42]}

clf = GridSearchCV(xgb_model, parameters, n_jobs=5, 
                   cv=KFold(n_splits=2, shuffle=True, random_state=42), 
                   scoring='neg_root_mean_squared_error',
                   verbose=0, refit=True)

clf.fit(X_train, y_train)

yt_pred = clf.predict(X_train)
yv_pred = clf.predict(X_valid)

print('Default Training RMSE: {:.3f}'.format(mean_squared_error(yt_pred, y_train, squared=False)))
print('Default Test RMSE: {:.3f}'.format(mean_squared_error(yv_pred, y_valid, squared=False)))


/usr/local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr

[19:31:39] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:573: 
Parameters: { "colsample_by*", "verbose_eval" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Training RMSE: 109.563
Test RMSE: 221.017


In [51]:
#Best Parameters Found
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

xgb_model = xgb.XGBRegressor(verbose_eval=50, seed =42)

parameters = {'objective':['reg:squarederror'],
              'eta': [0.03], #done
              'gamma':[0.1], #done
              'max_depth': [6], #done
              'min_child_weight':[1], #done
              'max_delta_step':[0], #done
              'subsample': [1], #done
              'sampling_method':['uniform'], #done
              'colsample_by*':[1], #done
              'n_estimators': [2000], #done
              'reg_alpha':[0], #done
              'reg_lambda': [1], #done
              'tree_method':['auto'], #done
              'max_leaves':[0], #done
              'num_parallel_tree':[1], #done
              'booster': ['gbtree'], #done
              'eval_metric': ['rmse'], #done
              'verbose_eval': [50], #not required
                'seed' :[42]} #not required

clf = GridSearchCV(xgb_model, parameters, n_jobs=5, 
                   cv=KFold(n_splits=2, shuffle=True, random_state=42), 
                   scoring='neg_root_mean_squared_error',
                   verbose=0, refit=True)

clf.fit(X_train, y_train)

yt_pred = clf.predict(X_train)
yv_pred = clf.predict(X_valid)

print('Training RMSE: {:.3f}'.format(mean_squared_error(yt_pred, y_train, squared=False)))
print('Test RMSE: {:.3f}'.format(mean_squared_error(yv_pred, y_valid, squared=False)))


/usr/local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/local/lib/python3.9/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/usr/local/lib/python3.9/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiInde

[23:30:33] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:573: 
Parameters: { "colsample_by*", "verbose_eval" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Training RMSE: 118.165
Test RMSE: 222.145


In [43]:
print(clf.best_params_)

print(clf.best_estimator_)

print(clf.best_score_)

print(clf.predict(df_test.copy()))


{'booster': 'gbtree', 'colsample_by*': 1, 'eta': 0.03, 'eval_metric': 'rmse', 'gamma': 0.1, 'max_delta_step': 0, 'max_depth': 6, 'max_leaves': 0, 'min_child_weight': 1, 'n_estimators': 2000, 'num_parallel_tree': 1, 'objective': 'reg:squarederror', 'reg_alpha': 0, 'reg_lambda': 1, 'sampling_method': 'uniform', 'seed': 42, 'subsample': 1, 'tree_method': 'auto', 'verbose_eval': 50}
XGBRegressor(base_score=0.5, booster='gbtree', colsample_by*=1,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             eta=0.03, eval_metric='rmse', gamma=0.1, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.0299999993, max_delta_step=0, max_depth=6,
             max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=2000, n_jobs=8,
             num_parallel_tree=1, random_state=42, reg_alpha=0, reg_lambda=1,
             sampling_method='uniform', scale_pos_weight=1, seed=42,
  

In [48]:
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

xgb_model = xgb.XGBRegressor(verbose_eval=50, seed =42)

parameters = {'objective':['reg:squarederror'],
              'eta': [0.03], #done
              'gamma':[0.1], #done
              'max_depth': [6], #done
              'min_child_weight':[1], #done
              'max_delta_step':[0], #done
              'subsample': [1], #done
              'sampling_method':['uniform'], #done
              'colsample_by*':[1], #done
              'n_estimators': [2000], #done
              'reg_alpha':[0], #done
              'reg_lambda': [1], #done
              'tree_method':['auto'], #done
              'max_leaves':[0], #done
              'num_parallel_tree':[1], #done
              'booster': ['gbtree'], #done
              'eval_metric': ['rmse'], #done
              'verbose_eval': [50], #not required
                'seed' :[42]} #not required

clf = GridSearchCV(xgb_model, parameters, n_jobs=5, 
                   cv=KFold(n_splits=2, shuffle=True, random_state=42), 
                   scoring='neg_root_mean_squared_error',
                   verbose=0, refit=True)

clf.fit(X_final, y_final)

/usr/local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/local/lib/python3.9/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/usr/local/lib/python3.9/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiInde

[21:28:17] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:573: 
Parameters: { "colsample_by*", "verbose_eval" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




GridSearchCV(cv=KFold(n_splits=2, random_state=42, shuffle=True),
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, gamma=None,
                                    gpu_id=None, importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monot...
                         'eta': [0.03], 'eval_metric': ['rmse'], 'gamma': [0.1],
                         'max_delta_step': [0], 'max_depth': [6],
                         'max_leaves': [0], 'min_child_weight': [1],
                         'n_estimators': [2000], 'num_parallel_tree': [1],
                         'objective':

In [49]:
yt_pred = clf.predict(X_train)
print('Training RMSE: {:.3f}'.format(mean_squared_error(yt_pred, y_train, squared=False)))


Training RMSE: 124.311


In [50]:
yTest = clf.predict(df_test)

yTest = yTest * df_test_size_sqft
yTest = pd.DataFrame(yTest.values, columns=['Predicted'], index=df_test.index)

yTest.to_csv('../predictions/xgboost_grid_search_final_full_db.csv')

[21:27:47] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:573: 
Parameters: { "colsample_by*", "verbose_eval" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[21:27:47] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:573: 
Parameters: { "colsample_by*", "verbose_eval" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


